In [1]:
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.tune.registry import get_trainable_cls, register_env
from robosuite.wrappers import GymWrapper
from common.reachEnv import * 
from common.envUtils import *
from gymnasium.envs.registration import register

# register(
#     id="ReachEnvUR5e-v0",
#     entry_point="common.envUtils:ReachEnvGym",  # 替换成你的模块路径
#     kwargs={'env_config' : None},
# )

# MultiAgentReach = make_multi_agent("ReachEnvUR5e-v0")

# register_env(
#     "env",
#     lambda _: MultiAgentReach(config={"num_agents": 2}),
# )

E0000 00:00:1753340618.310483   45727 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753340618.313951   45727 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753340618.323201   45727 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753340618.323216   45727 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753340618.323218   45727 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753340618.323219   45727 computation_placer.cc:177] computation placer already registered. Please check linka

In [ ]:
from ray import train, tune
import ray
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from common.envUtils import *

config = (
    SACConfig()
    .environment(env=ReachEnvGym)
    .training(
        initial_alpha=1.001,
        actor_lr=3e-5,
        critic_lr=3e-4,
        alpha_lr=1e-4,
        target_entropy="auto",
        n_step=1,
        tau=0.005,
        train_batch_size=256,
        target_network_update_freq=1,
        replay_buffer_config={
            "type": "PrioritizedEpisodeReplayBuffer",
            "capacity": 1000000,
            "alpha": 0.6,
            "beta": 0.4,
        },
        num_steps_sampled_before_learning_starts=256,
        model={
            "fcnet_hiddens": [256, 256],
            "fcnet_activation": "relu",
            "post_fcnet_hiddens": [],
            "post_fcnet_activation": None,
            "post_fcnet_weights_initializer": "orthogonal_",
            "post_fcnet_weights_initializer_config": {"gain": 0.01},
        },
    )
    .framework("torch")
    # .rl_module(
    #     model_config=DefaultModelConfig(
    #         fcnet_hiddens=[1024, 1024],
    #         fcnet_activation="relu",
    #         fcnet_kernel_initializer=nn.init.xavier_uniform_,
    #         head_fcnet_hiddens=[],
    #         head_fcnet_activation=None,
    #         head_fcnet_kernel_initializer="orthogonal_",
    #         head_fcnet_kernel_initializer_kwargs={"gain": 0.01},
    #     )
    # )
    .reporting(
        metrics_num_episodes_for_smoothing=5,
        min_sample_timesteps_per_iteration=1000,
    )
    .evaluation(
        evaluation_interval=1,
        evaluation_num_env_runners=1,
        evaluation_config={"seed": 42},
    )
    .env_runners(
        num_env_runners=4,             # Number of parallel processes (<= CPUs)
        num_envs_per_env_runner=2,     # Vectorized environments per process
        gym_env_vectorize_mode="ASYNC"
    )
)
# Build the Algorithm (PPO).
sac = config.build_algo()

E0000 00:00:1753341092.007975    5915 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753341092.022927    5915 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753341092.132423    5915 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753341092.132443    5915 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753341092.132445    5915 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753341092.132446    5915 computation_placer.cc:177] computation placer already registered. Please check linka

(SingleAgentEnvRunner pid=9502) [chatbus_1] 共享内存不存在，创建成功
(SingleAgentEnvRunner pid=9507) [chatbus_1] 共享内存已存在，连接成功


(SingleAgentEnvRunner pid=9507) /home/ey/anaconda3/envs/rlreach310/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
(SingleAgentEnvRunner pid=9507)   gym.logger.warn(
(SingleAgentEnvRunner pid=9507) /home/ey/anaconda3/envs/rlreach310/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
(SingleAgentEnvRunner pid=9507)   gym.logger.warn(
(SingleAgentEnvRunner pid=9502) 2025-07-24 15:11:45,992	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
2025-07-24 15:11:46,116	ERROR actor_manager.py:873 -- Ray error (The actor died because of an error raised in its creation task,

(SingleAgentEnvRunner pid=9503) [chatbus_1] 共享内存已存在，连接成功 [repeated 8x across cluster]


2025-07-24 15:11:54,711	INFO trainable.py:161 -- Trainable.setup took 17.566 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2025-07-24 15:11:54,712	WARNING util.py:61 -- Install gputil for GPU system monitoring.


(pid=10715) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=10715) E0000 00:00:1753341125.919335   10715 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=10715) E0000 00:00:1753341125.922704   10715 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=10715) W0000 00:00:1753341125.931920   10715 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
(pid=10715) W0000 00:00:1753341125.931946   10715 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
(pid=10715) W0000 00:00:1753341125.931947   10715 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid lin

(SingleAgentEnvRunner pid=10789) [chatbus_1] 共享内存已存在，连接成功


(SingleAgentEnvRunner pid=10790) /home/ey/anaconda3/envs/rlreach310/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
(SingleAgentEnvRunner pid=10790)   gym.logger.warn(
(SingleAgentEnvRunner pid=10790) /home/ey/anaconda3/envs/rlreach310/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
(SingleAgentEnvRunner pid=10790)   gym.logger.warn(
(pid=10788) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR [repeated 3x across cluster]
(pid=10788) E0000 00:00:1753341131.957872   10788 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered [repeated 3x across cluster]
(pid=10788) E0000 00:00:1753341131.961324   10788 cuda_blas.cc:1407] Unable to register cuBLAS factory: At

(SingleAgentEnvRunner pid=11078) [chatbus_1] 共享内存不存在，创建成功
(SingleAgentEnvRunner pid=10788) [chatbus_1] 共享内存已存在，连接成功 [repeated 11x across cluster]


(raylet) [2025-07-24 15:12:38,172 E 9425 9425] (raylet) node_manager.cc:3041: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 5e60799a1329c996a6e1790081232bfc71dafd4fd7f781ee77d682eb, IP: 192.168.202.162) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.202.162`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


In [2]:
# from pprint import pprint

# for _ in range(4):
#     pprint(sac.train())
from ray import train, tune

# Create a Tuner instance to manage the trials.
tuner = tune.Tuner(
    config.algo_class,
    param_space=config,
    # Specify a stopping criterion. Note that the criterion has to match one of the
    # pretty printed result metrics from the results returned previously by
    # ``.train()``. Also note that -1100 is not a good episode return for
    # Pendulum-v1, we are using it here to shorten the experiment time.
    run_config=train.RunConfig(
        stop={"env_runners/episode_return_mean": 0.0},
    ),
)
# Run the Tuner and capture the results.
results = tuner.fit()

2025-07-24 15:12:04,893	WARNING sac.py:487 -- You are running SAC on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-07-24 15:12:04,894	WARNING sac.py:487 -- You are running SAC on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-07-24 15:12:29,395	ERROR tune_controller.py:1331 -- Trial task failed for trial SAC_ReachEnvGym_80b15_00000
Traceback (most recent call last):
  File "/home/ey/anaconda3/envs/rlreach310/lib/pytho

In [ ]:
from ray import train, tune
from ray.rllib.algorithms.ppo import PPOConfig

config = (
    PPOConfig()
    .environment("Pendulum-v1")
    # Specify a simple tune hyperparameter sweep.
    .training(
        lr=tune.grid_search([0.001, 0.0005, 0.0001]),
    )
)

# Create a Tuner instance to manage the trials.
tuner = tune.Tuner(
    config.algo_class,
    param_space=config,
    # Specify a stopping criterion. Note that the criterion has to match one of the
    # pretty printed result metrics from the results returned previously by
    # ``.train()``. Also note that -1100 is not a good episode return for
    # Pendulum-v1, we are using it here to shorten the experiment time.
    run_config=train.RunConfig(
        stop={"env_runners/episode_return_mean": -1100.0},
    ),
)
# Run the Tuner and capture the results.
results = tuner.fit()